# Sentiment Analysis:

Naive Bayez works great on textual data.

### Work Flow:

1. Text Preprocessing: Remove Html tags, common words, remove special characeters.
2. Vectorize(BOW) the data.
3. Pass data to algorithm and train.
4. Deployement.



## Bag Of Words:

Find all unique words. For example we have following 4 words:-

Each new row represent a review and it value contains occurennce of that word in a review.

<table>

<tr>
<td>Great</td>
<td>awesome</td>
<td>poor</td>
<td>pathetic</td>
</tr>

<tr>
<td>3</td>
<td>1</td>
<td>0</td>
<td>0</td>
</tr>

<tr>
<td>0</td>
<td>1</td>
<td>2</td>
<td>4</td>
</tr>


</table>


Shape will become (50000, 4)

50000 -> Number of reviews

4 -> Number of unique words

## Apply bayes theorem now.

In [89]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import nltk # For StopWords
from nltk.corpus import stopwords # For Stopwords
from nltk.stem.porter import PorterStemmer # For stemming words
from sklearn.feature_extraction.text import CountVectorizer # For Vectorizing words
from sklearn.model_selection import train_test_split # For test train split
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv


# Model Implementation

In [47]:
data = pd.read_csv("/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv")
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [48]:
data = data.sample(10000).reset_index()
data.shape

(10000, 3)

In [49]:
# One Review
data['review'][0]

'"Gone With The Wind" is one of the most overrated movies in history. It is a film loved by mothers, grandmothers, and shut-ins who go to the movies once every five years. As a zombie movie, it blows. There isn\'t a shambling corpse in sight, and it\'s terribly light in the gore department. "Zombie 3", on the other hand, is big on shambling corpses and quite generous with its blood-spilling -- therefore, it is better than "Gone With the Wind". It is also not overrated. Most reviewers are under no delusions that it is rubbish. It is, however, not boring rubbish. After a terrorist steals a virus, he drops it while being pursued by a helicopter, and the chemicals leak into the ground. The terrorist, who has been exposed to the nasty concoction, hides in a hotel room where he slowly morphs into a flesh-eating zombie. One of his first victim\'s is a cleaning lady. Once she\'s bitten, Lucio Fulci\'s brand of hell breaks loose. As usual for a Fulci flick, the acting is atrocious, the storylin

## Text Cleaning:

1. Remove HTML Tags.
2. Remove special characters.
3. Convert everything to lower case.
4. Removing Stop Words.
5. Stemming (One words used in multiple form, example play, playing, played, etc)

In [50]:
# No missing value in data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   index      10000 non-null  int64 
 1   review     10000 non-null  object
 2   sentiment  10000 non-null  object
dtypes: int64(1), object(2)
memory usage: 234.5+ KB


In [51]:
# Changing Postive Negative to 0 and 1.
data['sentiment'].replace({'positive': 1, 'negative': 0}, inplace=True)

In [52]:
data.head()

,index,review,sentiment
0,7908,"""Gone With The Wind"" is one of the most overra...",0
1,9285,Wow! Why aren't more British movies like this....,1
2,16964,The British noble Sir Ronald Burton (Richard G...,1
3,36220,I agree with msinabottle; this is a great movi...,1
4,4740,Before I'd seen this movie I've heard a lot of...,1


### Removing HTML Tags

In [53]:
data.iloc[2].review

'The British noble Sir Ronald Burton (Richard Greene) decides to search his two best friends that have disappeared after visiting Count Karl von Bruno (Stephen MaNally), an evil and powerful man who lives in the Black Castle. Sir Burton travels undercover with another identity, since he fought against Count von Bruno in Afrika with his two missing friends and the count lost one eye in a battle. When he arrives in the castle, he is invited to hunt in the Black Forest around the castle with the count,.while he looks for evidences that the count has killed his friends. Later, he and the count\'s wife, Countess Elga von Bruno (Rita Corday), fall in love for each other and with the support of Dr. Meissen (Boris Karloff), Sir Burton and the countess try to escape from the claws of Count von Bruno. "The Black Castle" is an excellent movie from a romantic time, with action, romance, mystery and even horror. The story is gripping, and is a great entertainment for any audience. My vote is nine.<

In [54]:
# Removing all HTML Tags
clean = re.compile('<.*?>')
re.sub(clean, '', data.iloc[2]. review)

'The British noble Sir Ronald Burton (Richard Greene) decides to search his two best friends that have disappeared after visiting Count Karl von Bruno (Stephen MaNally), an evil and powerful man who lives in the Black Castle. Sir Burton travels undercover with another identity, since he fought against Count von Bruno in Afrika with his two missing friends and the count lost one eye in a battle. When he arrives in the castle, he is invited to hunt in the Black Forest around the castle with the count,.while he looks for evidences that the count has killed his friends. Later, he and the count\'s wife, Countess Elga von Bruno (Rita Corday), fall in love for each other and with the support of Dr. Meissen (Boris Karloff), Sir Burton and the countess try to escape from the claws of Count von Bruno. "The Black Castle" is an excellent movie from a romantic time, with action, romance, mystery and even horror. The story is gripping, and is a great entertainment for any audience. My vote is nine.T

In [55]:
# Function to clean HTML Tags:
def clean_html(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

In [56]:
data['review'] = data['review'].apply(clean_html)

In [57]:
data.iloc[2].review

'The British noble Sir Ronald Burton (Richard Greene) decides to search his two best friends that have disappeared after visiting Count Karl von Bruno (Stephen MaNally), an evil and powerful man who lives in the Black Castle. Sir Burton travels undercover with another identity, since he fought against Count von Bruno in Afrika with his two missing friends and the count lost one eye in a battle. When he arrives in the castle, he is invited to hunt in the Black Forest around the castle with the count,.while he looks for evidences that the count has killed his friends. Later, he and the count\'s wife, Countess Elga von Bruno (Rita Corday), fall in love for each other and with the support of Dr. Meissen (Boris Karloff), Sir Burton and the countess try to escape from the claws of Count von Bruno. "The Black Castle" is an excellent movie from a romantic time, with action, romance, mystery and even horror. The story is gripping, and is a great entertainment for any audience. My vote is nine.T

### Converting everything to lower

In [58]:
def convert_lower(text):
    return text.lower()

In [59]:
data['review'] = data['review'].apply(convert_lower)

In [60]:
data.head()

,index,review,sentiment
0,7908,"""gone with the wind"" is one of the most overra...",0
1,9285,wow! why aren't more british movies like this....,1
2,16964,the british noble sir ronald burton (richard g...,1
3,36220,i agree with msinabottle; this is a great movi...,1
4,4740,before i'd seen this movie i've heard a lot of...,1


### Remove Special Characters

In [61]:
def remove_special(text):
    x = ""
    
    for i in text:
        if i.isalnum():
            x = x + i
        else:
            x = x + " "
    return x

In [62]:
data.iloc[2].review

'the british noble sir ronald burton (richard greene) decides to search his two best friends that have disappeared after visiting count karl von bruno (stephen manally), an evil and powerful man who lives in the black castle. sir burton travels undercover with another identity, since he fought against count von bruno in afrika with his two missing friends and the count lost one eye in a battle. when he arrives in the castle, he is invited to hunt in the black forest around the castle with the count,.while he looks for evidences that the count has killed his friends. later, he and the count\'s wife, countess elga von bruno (rita corday), fall in love for each other and with the support of dr. meissen (boris karloff), sir burton and the countess try to escape from the claws of count von bruno. "the black castle" is an excellent movie from a romantic time, with action, romance, mystery and even horror. the story is gripping, and is a great entertainment for any audience. my vote is nine.t

In [63]:
remove_special(data.iloc[2].review)

'the british noble sir ronald burton  richard greene  decides to search his two best friends that have disappeared after visiting count karl von bruno  stephen manally   an evil and powerful man who lives in the black castle  sir burton travels undercover with another identity  since he fought against count von bruno in afrika with his two missing friends and the count lost one eye in a battle  when he arrives in the castle  he is invited to hunt in the black forest around the castle with the count  while he looks for evidences that the count has killed his friends  later  he and the count s wife  countess elga von bruno  rita corday   fall in love for each other and with the support of dr  meissen  boris karloff   sir burton and the countess try to escape from the claws of count von bruno   the black castle  is an excellent movie from a romantic time  with action  romance  mystery and even horror  the story is gripping  and is a great entertainment for any audience  my vote is nine ti

In [64]:
data['review'] = data['review'].apply(remove_special)
data.head()

,index,review,sentiment
0,7908,gone with the wind is one of the most overra...,0
1,9285,wow why aren t more british movies like this ...,1
2,16964,the british noble sir ronald burton richard g...,1
3,36220,i agree with msinabottle this is a great movi...,1
4,4740,before i d seen this movie i ve heard a lot of...,1


### Remove Stop Words

In [65]:
stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [66]:
def remove_stopwords(text): 
    x = []
    for i in text.split():
        if i not in stopwords.words('english'):
            x.append(i)
    y = x[:]
    x.clear()
    return y

In [67]:
data.iloc[2].review

'the british noble sir ronald burton  richard greene  decides to search his two best friends that have disappeared after visiting count karl von bruno  stephen manally   an evil and powerful man who lives in the black castle  sir burton travels undercover with another identity  since he fought against count von bruno in afrika with his two missing friends and the count lost one eye in a battle  when he arrives in the castle  he is invited to hunt in the black forest around the castle with the count  while he looks for evidences that the count has killed his friends  later  he and the count s wife  countess elga von bruno  rita corday   fall in love for each other and with the support of dr  meissen  boris karloff   sir burton and the countess try to escape from the claws of count von bruno   the black castle  is an excellent movie from a romantic time  with action  romance  mystery and even horror  the story is gripping  and is a great entertainment for any audience  my vote is nine ti

In [68]:
remove_stopwords(data.iloc[2].review)

['british',
 'noble',
 'sir',
 'ronald',
 'burton',
 'richard',
 'greene',
 'decides',
 'search',
 'two',
 'best',
 'friends',
 'disappeared',
 'visiting',
 'count',
 'karl',
 'von',
 'bruno',
 'stephen',
 'manally',
 'evil',
 'powerful',
 'man',
 'lives',
 'black',
 'castle',
 'sir',
 'burton',
 'travels',
 'undercover',
 'another',
 'identity',
 'since',
 'fought',
 'count',
 'von',
 'bruno',
 'afrika',
 'two',
 'missing',
 'friends',
 'count',
 'lost',
 'one',
 'eye',
 'battle',
 'arrives',
 'castle',
 'invited',
 'hunt',
 'black',
 'forest',
 'around',
 'castle',
 'count',
 'looks',
 'evidences',
 'count',
 'killed',
 'friends',
 'later',
 'count',
 'wife',
 'countess',
 'elga',
 'von',
 'bruno',
 'rita',
 'corday',
 'fall',
 'love',
 'support',
 'dr',
 'meissen',
 'boris',
 'karloff',
 'sir',
 'burton',
 'countess',
 'try',
 'escape',
 'claws',
 'count',
 'von',
 'bruno',
 'black',
 'castle',
 'excellent',
 'movie',
 'romantic',
 'time',
 'action',
 'romance',
 'mystery',
 'even',

In [69]:
data['review'] = data['review'].apply(remove_stopwords)

### Perform Stemming

In [70]:
ps = PorterStemmer()

In [71]:
y = []
def stem_words(text):
    for i in text:
        y.append(ps.stem(i))
    z = y[:]
    y.clear()
    return z

In [72]:
data['review'] = data['review'].apply(stem_words)
data.head()

,index,review,sentiment
0,7908,"[gone, wind, one, overr, movi, histori, film, ...",0
1,9285,"[wow, british, movi, like, great, right, passa...",1
2,16964,"[british, nobl, sir, ronald, burton, richard, ...",1
3,36220,"[agre, msinabottl, great, movi, dialogu, snipp...",1
4,4740,"[seen, movi, heard, lot, prais, quit, mani, ex...",1


In [73]:
# Join Back
def join_back(list_input):
    return " ".join(list_input)

In [74]:
data['review'] = data['review'].apply(join_back)
data.head()

,index,review,sentiment
0,7908,gone wind one overr movi histori film love mot...,0
1,9285,wow british movi like great right passag money...,1
2,16964,british nobl sir ronald burton richard green d...,1
3,36220,agre msinabottl great movi dialogu snippet rai...,1
4,4740,seen movi heard lot prais quit mani exclam hor...,1


## Final Preparations:

To get all words used in review

In [75]:
cv= CountVectorizer()

In [76]:
x = cv.fit_transform(data['review']).toarray()

In [77]:
x.shape

(10000, 36394)

In [78]:
y = data.iloc[:,-1].values

In [79]:
y.shape

(10000,)

In [80]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)

In [81]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(8000, 36394)
(8000,)
(2000, 36394)
(2000,)


### Preparing model

In [82]:
clf1 = GaussianNB()
clf2 = MultinomialNB()
clf3 = BernoulliNB()

In [83]:
clf1.fit(x_train, y_train)
clf2.fit(x_train, y_train)
clf3.fit(x_train, y_train)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [84]:
y_pred1 = clf1.predict(x_test)
y_pred2 = clf2.predict(x_test)
y_pred3 = clf3.predict(x_test)

In [86]:
y_pred1.shape

(2000,)

In [92]:
print("Gaussian:", accuracy_score(y_test, y_pred1))
print("Multinomial:", accuracy_score(y_test, y_pred2))
print("Bernaulli:", accuracy_score(y_test, y_pred3))

Gaussian: 0.6155
